# Exercise 7 - an analysis of SPY monthly calls in 2018

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_trade = pd.read_csv('../data/spy_2018_call_trade.csv')
df_pnl = pd.read_csv('../data/spy_2018_call_pnl.csv')

In [3]:
df_trade.head()

,execution_date,direction,quantity,underlying,type,strike,expiration,d2x,trade_price
0,2017-12-15,sell,1,SPY,call,270,2018-01-19,22,1.14
1,2018-01-19,sell,1,SPY,call,284,2018-02-16,20,1.27
2,2018-02-16,sell,1,SPY,call,278,2018-03-16,19,1.95
3,2018-03-16,sell,1,SPY,call,280,2018-04-20,24,1.57
4,2018-04-20,sell,1,SPY,call,272,2018-05-18,20,1.71


#### Chek that there are a total of 12 options represented in df_trade.  Recall that an option is uniquely defined by its `underlying`, `type`, `strike`, and `expiration`.

In [4]:
df_trade.shape

(12, 9)

In [47]:
# checking the number of unique options in df_trade
df_trade.groupby(['underlying', 'type', 'strike', 'expiration']).size().reset_index().shape

(12, 5)

In [6]:
df_pnl.head()

,underlying,upx,type,expiration,data_date,strike,bid,ask,implied_vol,delta,dly_opt_pnl,dly_dh_pnl
0,SPY,266.529999,call,2018-01-19,2017-12-15,270,1.14,1.16,0.068257,0.328344,-0.02,0.000000
1,SPY,268.230011,call,2018-01-19,2017-12-18,270,1.68,1.69,0.071450,0.421353,-0.53,0.558189
2,SPY,267.250000,call,2018-01-19,2017-12-19,270,1.39,1.41,0.074841,0.365808,0.28,-0.412931
3,SPY,267.100006,call,2018-01-19,2017-12-20,270,1.10,1.11,0.070911,0.327058,0.30,-0.054869
4,SPY,267.540009,call,2018-01-19,2017-12-21,270,1.31,1.32,0.072183,0.372113,-0.21,0.143906


In [48]:
df_pnl.groupby(['underlying', 'type', 'strike', 'expiration']).size().reset_index().shape

(12, 5)

In [8]:
# separating out execution dates
df_execution = df_trade[['expiration', 'execution_date']]

# joining df_pnl and df_execution to add execution date to df_pnl
df_pnl = \
    pd.merge(df_pnl, df_execution, on=['expiration'])

In [12]:
df_pnl[df_pnl.execution_date == df_pnl.data_date]['delta'].describe()

count    12.000000
mean      0.307980
std       0.014992
min       0.283518
25%       0.296913
50%       0.306119
75%       0.318692
max       0.330158
Name: delta, dtype: float64

In [16]:
df_premium = \
    df_pnl[df_pnl.execution_date == df_pnl.data_date][['expiration', 'bid']]
df_premium.head()

,expiration,bid
0,2018-01-19,1.14
23,2018-02-16,1.27
44,2018-03-16,1.95
64,2018-04-20,1.57
89,2018-05-18,1.71


In [25]:
df_pnl_expiration = \
    df_pnl.groupby(['expiration'])['dly_opt_pnl'].agg([np.sum]).reset_index()

df_pnl_expiration.rename(columns={'sum':'exp_pnl'}, inplace=True)

df_pnl_expiration.head()

,expiration,exp_pnl
0,2018-01-19,-9.250015
1,2018-02-16,1.270000
2,2018-03-16,1.950000
3,2018-04-20,1.570000
4,2018-05-18,1.710000


In [29]:
df_prem_vs_pnl = \
    pd.merge(df_premium, df_pnl_expiration, on='expiration')

df_prem_vs_pnl['pnl_ratio'] = df_prem_vs_pnl.exp_pnl / df_prem_vs_pnl.bid

df_prem_vs_pnl

,expiration,bid,exp_pnl,pnl_ratio
0,2018-01-19,1.14,-9.250015,-8.114048
1,2018-02-16,1.27,1.270000,1.000000
2,2018-03-16,1.95,1.950000,1.000000
3,2018-04-20,1.57,1.570000,1.000000
4,2018-05-18,1.71,1.710000,1.000000
5,2018-06-15,1.46,-0.569999,-0.390410
6,2018-07-20,1.48,1.480000,1.000000
7,2018-08-17,1.35,-0.669989,-0.496288
8,2018-09-21,1.39,-1.650009,-1.187057
9,2018-10-19,1.15,1.150000,1.000000


In [31]:
# delta-hedge pnl is negatively correlated with the (probably should do this after it's into daily values)
np.corrcoef(df_pnl.dly_opt_pnl, df_pnl.dly_dh_pnl)

array([[ 1.        , -0.92697748],
       [-0.92697748,  1.        ]])

In [34]:
# creating the total PNL column
df_pnl['dly_tot_pnl'] = df_pnl.dly_opt_pnl + df_pnl.dly_dh_pnl

In [39]:
# aggregating to get daily total
df_comparison = \
    df_pnl.groupby(['data_date']).agg({'dly_opt_pnl':np.sum, 'dly_tot_pnl':np.sum}).reset_index()

df_comparison.head()

,data_date,dly_opt_pnl,dly_tot_pnl
0,2017-12-15,-0.02,-0.020000
1,2017-12-18,-0.53,0.028189
2,2017-12-19,0.28,-0.132931
3,2017-12-20,0.30,0.245131
4,2017-12-21,-0.21,-0.066094


In [45]:
# comparing standard deviation of PNLs
print('naked options: ', np.std(df_comparison.dly_opt_pnl))
print('delta hedged: ', np.std(df_comparison.dly_tot_pnl))

naked options:  0.581733284292837
delta hedged:  0.2373096110672859
